In [15]:
from fastembed import SparseTextEmbedding

documents = [
    "Кто ты?",
    "Кто я сложжж?",
]

model = SparseTextEmbedding(model_name="Qdrant/bm25")
embeddings = list(model.embed(documents))
embeddings

[SparseEmbedding(values=array([1.68320383, 1.68320383]), indices=array([ 252671796, 1907436970])),
 SparseEmbedding(values=array([1.67868852, 1.67868852, 1.67868852]), indices=array([252671796, 740001907, 760805627]))]

In [17]:
from FlagEmbedding import BGEM3FlagModel

model = BGEM3FlagModel("BAAI/bge-m3", use_fp16=False)

Fetching 30 files:   0%|          | 0/30 [00:00<?, ?it/s]

/home/harddisk/projects/legal_ml/.venv/lib/python3.10/site-packages/FlagEmbedding/BGE_M3/modeling.py:335: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  colbert_state_dict = 

In [35]:
model.encode("Как мне получить социальные выплаты?", return_sparse=True, return_colbert_vecs=True)

{'dense_vecs': array([-0.04810571,  0.01629228, -0.0242204 , ..., -0.01850992,
        -0.0539299 , -0.03781377], dtype=float32),
 'lexical_weights': defaultdict(int,
             {'5187': 0.13655508,
              '4042': 0.15718421,
              '28068': 0.1956503,
              '207964': 0.22945362,
              '138417': 0.25035983,
              '32': 0.034747005}),
 'colbert_vecs': array([[-0.03758522,  0.02490063, -0.0064325 , ...,  0.03040174,
          0.05951791, -0.02600569],
        [-0.01922581,  0.01614589, -0.00853095, ...,  0.02471888,
          0.03960711, -0.0021224 ],
        [-0.02498882,  0.03419046,  0.01096246, ...,  0.02128083,
          0.0368477 , -0.01440094],
        ...,
        [-0.04475792,  0.02650905,  0.02954892, ...,  0.0380312 ,
          0.04102869,  0.00359551],
        [-0.04123949,  0.00559941, -0.00024967, ...,  0.03513594,
          0.06965889, -0.00638408],
        [-0.00274066,  0.01769873, -0.03015449, ...,  0.02136883,
          0.0270129

In [37]:
model.convert_id_to_token(
    [
        {
            "5187": 0.13655508,
            "4042": 0.15718421,
            "28068": 0.1956503,
            "207964": 0.22945362,
            "138417": 0.25035983,
            "32": 0.034747005,
        }
    ]
)

{'Как': 0.13655508,
 'мне': 0.15718421,
 'получить': 0.1956503,
 'социальные': 0.22945362,
 'выплаты': 0.25035983,
 '?': 0.034747005}

https://qdrant.tech/articles/hybrid-search/

https://qdrant.tech/documentation/concepts/vectors/

Создаем векторную базу данных с несколькими векторами

In [56]:
from qdrant_client import QdrantClient, models

client = QdrantClient(":memory:")

col_name = "hybrid_search"

In [57]:
client.create_collection(
    collection_name=col_name,
    vectors_config={
        "dence": models.VectorParams(size=100, distance=models.Distance.COSINE),
        "colbert": models.VectorParams(
            size=128,
            distance=models.Distance.COSINE,
            multivector_config=models.MultiVectorConfig(
                comparator=models.MultiVectorComparator.MAX_SIM
            ),
        ),
    },
    sparse_vectors_config={"text-sparse": models.SparseVectorParams()},
)

ValidationError: 6 validation errors for CreateCollection
vectors.VectorParams.size
  Field required [type=missing, input_value={'dence': VectorParams(si...r.MAX_SIM: 'max_sim'>))}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.10/v/missing
vectors.VectorParams.distance
  Field required [type=missing, input_value={'dence': VectorParams(si...r.MAX_SIM: 'max_sim'>))}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.10/v/missing
vectors.VectorParams.dence
  Extra inputs are not permitted [type=extra_forbidden, input_value=VectorParams(size=100, di...multivector_config=None), input_type=VectorParams]
    For further information visit https://errors.pydantic.dev/2.10/v/extra_forbidden
vectors.VectorParams.text-sparse
  Extra inputs are not permitted [type=extra_forbidden, input_value=SparseVectorParams(index=None, modifier=None), input_type=SparseVectorParams]
    For further information visit https://errors.pydantic.dev/2.

In [52]:
client.scroll(collection_name=col_name)

([], None)